In [11]:
import os
from google.colab import drive
import os, shutil
from tqdm import tqdm
from PIL import Image
import torch
import torchvision
from torchvision.datasets import CIFAR10
import os, wandb

In [12]:
!pip -q install torch torchvision ruamel.yaml wandb pillow tqdm
# Optional: silence W&B if you don’t want to log

os.environ["WANDB_DISABLED"] = "true"

Generic Folder Layout:
My Drive/

└── Colab Notebooks/

    └── Diffusion_IDL_FinalProjectF25/

        ├── runColabDiff.ipynb
        └── F25-Deep-Learning-Project.zip  (Folder with files and executables)


In [13]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Unzip the file and extract the key project files
# Path to your .zip file inside Drive
zip_path = "/content/drive/My Drive/Colab Notebooks/Diffusion_IDL_FinalProjectF25/F25-Deep-Learning-Project.zip"

# Unzip it into the Colab working directory
!unzip -q "$zip_path" -d /content/project

# Go into the project directory (adjust name if needed)
%cd /content/project/F25-Deep-Learning-Project
!ls -al


replace /content/project/F25-Deep-Learning-Project/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
/content/project/F25-Deep-Learning-Project
total 108
drwxr-xr-x 11 root root  4096 Oct 30 03:21 .
drwxr-xr-x  4 root root  4096 Oct 30 03:19 ..
drwxr-xr-x  2 root root  4096 Oct 30 03:21 configs
-rw-r--r--  1 root root 10244 Oct 29 15:31 .DS_Store
drwxr-xr-x  7 root root  4096 Oct 30 03:21 .git
drwxr-xr-x  2 root root  4096 Oct 30 03:21 image_folder
-rw-r--r--  1 root root  3830 Oct 29 16:10 inference.py
-rw-r--r--  1 root root     0 Oct 16 16:00 __init__.py
-rw-r--r--  1 root root 11357 Oct 16 16:00 LICENSE
drwxr-xr-x  3 root root  4096 Oct 30 03:21 models
drwxr-xr-x  3 root root  4096 Oct 30 03:21 pipelines
-rw-r--r--  1 root root  3641 Oct 29 15:07 README_FilesWalkthrough.md
-rw-r--r--  1 root root  1694 Oct 27 20:01 README.md
-rw-r--r--  1 root root    38 Oct 16 16:00 requirements.txt
drwxr-xr-x  3 root root  4096 Oct 30 03:21 schedulers
drwxr-xr-x  3 root root  4096 Oct 26 16:50 t

In [15]:
# Download and distribute CIFAR10 data into train and test
root = "/content/data"
out_root = f"{root}/cifar10_imagefolder"
os.makedirs(root, exist_ok=True)

In [16]:
# Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009.
# Tech Report: https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf
# Dataset link (original not torchvision): https://www.cs.toronto.edu/~kriz/cifar.html
train_ds = CIFAR10(root=root, train=True,  download=True)
test_ds  = CIFAR10(root=root, train=False, download=True)
classes = train_ds.classes  # ['airplane','automobile',...,'truck']

In [17]:
#  export to ImageFolder: /content/data/cifar10_imagefolder/{train|val}/{class}/img_XXXXX.png
def dump_split(ds, split_name):
    split_dir = f"{out_root}/{split_name}"
    if os.path.exists(split_dir):
        shutil.rmtree(split_dir)
    os.makedirs(split_dir, exist_ok=True)
    # make per-class dirs
    for c in classes:
        os.makedirs(os.path.join(split_dir, c), exist_ok=True)
    # write PNGs
    for i in tqdm(range(len(ds)), desc=f"Writing {split_name} set"):
        img, label = ds[i]                       # PIL Image, int label
        cls = classes[label]
        img.save(os.path.join(split_dir, cls, f"img_{i:05d}.png"))

dump_split(train_ds, "train")
dump_split(test_ds,  "val")

print("Done. Point train.py --data_dir to:", f"{out_root}/train")

Writing val set: 100%|██████████| 10000/10000 [00:03<00:00, 2730.56it/s]

Done. Point train.py --data_dir to: /content/data/cifar10_imagefolder/train


In [18]:
print("CUDA:", torch.cuda.is_available())


CUDA: True


In [19]:
os.environ["WANDB_ENTITY"] = "F25_Deep_Learning"
os.environ["WANDB_PROJECT"] = "Diffusion_IDLF25_FinalProject"
os.environ["WANDB_API_KEY"] = "ddd2e2b4711784c184999cb5f1cba19977695095"
wandb.login()

True

In [20]:
# TRAINING: RUN train.py
%cd /content/project/F25-Deep-Learning-Project


!mkdir -p "/content/drive/My Drive/Colab Notebooks/Diffusion_IDL_FinalProjectF25/experiments"
!python train.py \
  --output_dir "/content/drive/My Drive/Colab Notebooks/Diffusion_IDL_FinalProjectF25/experiments" \
  --data_dir /content/data/cifar10_imagefolder/train \
  --val_dir  /content/data/cifar10_imagefolder/val \
  --image_size 32 --unet_in_size 32 \
  --num_classes 10 \
  --batch_size 64 --num_workers 8 \
  --num_epochs 50 --learning_rate 1e-4 --weight_decay 1e-4 \
  --num_train_timesteps 1000 --num_inference_steps 50 \
  --beta_start 0.0001 --beta_end 0.02 --beta_schedule linear \
  --use_ddim True --ddim_eta 0.0

/content/project/F25-Deep-Learning-Project
10/30/2025 03:22:19 - INFO - __main__ - Training with a single process on 1 device (cuda).
10/30/2025 03:22:19 - INFO - __main__ - Creating dataset
10/30/2025 03:22:19 - INFO - __main__ - Validation set found at /content/data/cifar10_imagefolder/val with 10000 images.
10/30/2025 03:22:19 - INFO - __main__ - Creating model
10/30/2025 03:22:20 - INFO - __main__ - Number of parameters: 38.38M
wandb: Currently logged in as: ssk-phl (F25_Deep_Learning) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ setting up run eqors4v6 (0.1s)
wandb: ⣽ setting up run eqors4v6 (0.1s)
wandb: ⣾ setting up run eqors4v6 (0.1s)
wandb: Tracking run with wandb version 0.22.2
wandb: Run data is saved locally in /content/project/F25-Deep-Learning-Project/wandb/run-20251030_032220-eqors4v6
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run exp-2
wandb: ⭐️ View project at https://wandb.ai/F2